## Resampling (Remuestreo)

El remuestreo es una metodología de uso económico de una muestra de datos para mejorar la precisión y cuantificar la incertidumbre de un parámetro de población.

El remuestreo es una herramienta que nos permite:

- Seleccionar una muestra de una población. 
- entrenar un modelo contra la muestra.
- verificar el rendimiento del modelo.

El remuestreo ha sido considerado como una tarea “computacionalmente costosa”, sin embargo con los recursos computacionales disponibles, cada vez se hace más factible ejecutarlo tanto en “commodity laptops” como en el “cloud”.

Existen diversas técnicas, sin embargo las más populares son:

- Selección aleatoria
- Validación Cruzada (Cross-Validation)
- Bootstrap

A continuación, vamos a ver como se puede utilizar cada uno de estos métodos, para la evaluación de modelos de aprendizaje automático. 

### Selección Aleatoria

La selección aleatoria busca dividir el dataset en dos partes: el set de entrenamiento (training set) y un set de pruebas (test set). 

Esta técnica es particularmente útil y rápida de ejecutar en donde se definen una partición de los datos para entrenamiento y otra para prueba. Normalmente se escoge una muestra del 80% de los datos para entrenar el modelo y un 20% para probarlos. Estos porcentajes pueden variar dependiendo del tamaño del dataset. 

Existe un inconveniente respecto a la selección de una única muestra aleatoria; está reside en que los valores que queden tanto en el training set como en el test set no sean verdaderamente representativos de la población.

Para este notebook vamos a utilizar el dataset Social Network Ads, donde vamos predecir si alguien va a realizar una compra. 

- Variable de Respuesta: Purchased (1/0)
- Variables Independientes: Gender (Male/Female), Age (numerica), EstimatedSalary (numerica)

In [ ]:
import pandas as pd
import numpy as np

data = pd.read_csv("data/Social_Network_Ads.csv")

y = np.array(data["Purchased"]).reshape(-1,1)

data.drop(columns=['User ID','Purchased'],axis=1,inplace=True)
data.head()

data["Gender"] =  np.array([1 if y == 'Male' else 0 for y in data["Gender"]])

X = data

print("total de rows en el dataset:", X.shape[0])

data.head()

In [ ]:
# seleccion de muestra aleatoria con SkLearn (ratio 80/20)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

print("Tamaño del set de Entrenamiento", X_train.shape)
print("Tamaño del set de Pruebas", X_test.shape)

In [ ]:
# Prueba con Logistic Regression
from sklearn.linear_model import LogisticRegression

# seleccionamos la particion de entrenamiento, para entrenar el modelo
model = LogisticRegression(random_state=0).fit(X_train, y_train.reshape(-1))

# probabilidad promedio del subset.
acc_train = model.score(X_train, y_train.reshape(-1))
acc_test = model.score(X_test, y_test.reshape(-1))

print("Exactitud de la prediccion (Train):", acc_train)
print("Exactitud de la prediccion (Test):", acc_test)

### Validación Cruzada (Cross-Validation)

Validación cruzada es la técnica de evaluación de modelos de ML donde se busca obtener varias muestras del training set y del test set de un mismo dataset. A diferencia de la tecnica de selección aleatoria, aquí se pueden particionar el test-set y el training set multiples veces con el objetivo de obtener un  rendimiento promedio.

Existen diferentes alternativas para realizar esto:

- **LOOCV** (Leave-one-out Cross-Validation): el test set se compone solamente de un row (1) y el training set de el resto de los datos. Esto se repite n cantidad de veces y se promedian al final las exactitudes reportadas. 
- **K-Fold CV** (K-Fold Cross-Validation): Esta técnica es menos exhaustiva que LOOCV y se particiona el dataset en K test-sets. Por ejemplo, si usamos 10-Fold CV, se particiona el dataset en 10 partes distintas, donde cada parte sirve de test-set y el resto de los datos de training set. Esto permite probar 10 configuraciones de remuestreo diferentes, pero sin tener problemas de rendimiento como en el caso de LOOCV. Se dice que LOOCV es K-Fold CV cuando K=N

ejemplo de K-Fold CV (k=4):

<img src="img/cv.png"/>

Nuestra recomendación es que se use K = 5, 10 o 20 según la cantidad de datos. 

Vamos a utilizar 10-fold CV con nuestro dataset de Social Network Ads: 


In [ ]:
from sklearn.model_selection import cross_val_score

model = LogisticRegression(random_state=0)
scores = cross_val_score(model, X, y.reshape(-1), cv=10)

print("Exactitud de cada particion:", scores)
print("Exactitud Promedio:", scores.mean())

### Bootstrap

El método bootstrap es una técnica estadística para estimar cantidades sobre una población promediando estimaciones de múltiples muestras de datos pequeños.

Como escoger la muestra:
- Determine el tamano de la muestra = Tm
- Mientras n < Tm
    - Seleccione un row aleatorio del dataset
    - agreguelo a la muestra

Los elementos que queden fuera de la muestra se conocen como "ejemplos fuera de la muestra" (OOB - Out-of-Bag Samples)

Tamano del Sample:  entre 50% y 80% del dataset, esto puede variar dependiendo de tamano.
Repeticiones: de 20 a 50, sin embargo entre mas repeticiones, mejor. 

In [ ]:
# se usa el metodo resample para seleccionar la muestra
from sklearn.utils import resample
from sklearn.metrics import accuracy_score

data = pd.read_csv("data/Social_Network_Ads.csv")
data.drop(columns=['User ID'],axis=1,inplace=True)
data["Gender"] =  np.array([1 if y == 'Male' else 0 for y in data["Gender"]])

# tamano de la  muestra (50%)
Tm = int(data.shape[0]/2) 
# cuantas  veces vamos a repetir el proceso 
reps = 20
#
scores = np.array([]);

for i in np.arange(1,reps):
    # seleccion de los samples
    boot = resample(data, replace=False, n_samples=Tm)
    oob = data.drop(boot.index, axis=0)
    y_train = boot["Purchased"]
    X_train = boot.loc[:, boot.columns != 'Purchased']
    y_test = oob["Purchased"]
    X_test = oob.loc[:, oob.columns != 'Purchased']

    # seleccionamos la particion de entrenamiento, para entrenar el modelo
    model = LogisticRegression(random_state=0).fit(X_train, np.array(y_train).reshape(-1))

    # probabilidad promedio del subset.
    score = model.score(X_test, np.array(y_test).reshape(-1))
    scores = np.append(scores, score)

print("Exactitud Promedio:", scores.mean())